In [2]:
import zipfile
import os

zip_path = "DFU_dataset.zip"
extract_to = "DFU_dataset"

# Extract only if not already done
if not os.path.exists(extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("✅ Dataset extracted to:", extract_to)
else:
    print("📂 Dataset already extracted.")


📂 Dataset already extracted.


In [4]:
import os

nested = "DFU_dataset/DFU_dataset"
renamed = "DFU_dataset/data"

if os.path.exists(nested):
    os.rename(nested, renamed)
    print("✅ Folder renamed to 'data'")
else:
    print("✅ No rename needed or already renamed.")


✅ No rename needed or already renamed.


In [5]:
dataset_dir = "DFU_dataset/data"

In [7]:
pip install tensorflow


  Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached grpcio-1.73.1-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.10.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached ml_dtypes-0.5.1-cp312-cp312-win_amd64.whl.metadata (22 kB)
  Using cached namex-0.1.0-py3-none-any.whl.metadata (322 bytes)
  Using cached optree-0.16.0-cp312-cp312-win_amd64.whl.metadata (31 kB)
  Using cached te

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.preprocessing.image import load_img, img_to_array

img_size = 224
mobilenet = MobileNetV2(weights='imagenet', include_top=False, pooling='avg', input_shape=(img_size, img_size, 3))
resnet = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(img_size, img_size, 3))

In [9]:
def extract_features(folder, label):
    features, labels = [] , []
    for fname in os.listdir(folder):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(folder, fname)
            img = load_img(img_path, target_size=(img_size, img_size))
            arr = img_to_array(img)
            arr = np.expand_dims(arr, axis=0)

            f1 = mobilenet.predict(mobilenet_preprocess(arr.copy()), verbose=0)
            f2 = resnet.predict(resnet_preprocess(arr.copy()), verbose=0)
            fused = np.concatenate([f1.flatten(), f2.flatten()])

            features.append(fused)
            labels.append(label)
    return features, labels


In [15]:
dataset_dir = r"C:\Users\veni\DFU_dataset"


In [17]:
for cls in ["Healthy foot", "Ulcered foot"]:
    for phase in ["train", "test"]:
        folder = os.path.join(dataset_dir, phase, cls)
        print("🔍 Checking folder:", folder)
        print("Exists:", os.path.exists(folder))


🔍 Checking folder: C:\Users\veni\DFU_dataset\train\Healthy foot
Exists: True
🔍 Checking folder: C:\Users\veni\DFU_dataset\test\Healthy foot
Exists: True
🔍 Checking folder: C:\Users\veni\DFU_dataset\train\Ulcered foot
Exists: True
🔍 Checking folder: C:\Users\veni\DFU_dataset\test\Ulcered foot
Exists: True


In [18]:
X, y = [], []

for cls in ["Healthy foot", "Ulcered foot"]:
    for phase in ["train", "test"]:
        folder = os.path.join(dataset_dir, phase, cls)
        label = "healthy" if "Healthy" in cls else "ulcer"
        ftrs, lbls = extract_features(folder, label)
        X.extend(ftrs)
        y.extend(lbls)

X = np.array(X)
y = np.array(y)

In [19]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

svm = SVC(kernel='rbf', probability=True)
params = {'C': [1, 10, 100], 'gamma': [0.001, 0.01, 0.1]}

grid = GridSearchCV(svm, params, cv=3, scoring='accuracy')
grid.fit(X_scaled, y_encoded)

print(f"✅ Best Parameters: {grid.best_params_}")
print(f"✅ Best CV Accuracy: {grid.best_score_:.4f}")

✅ Best Parameters: {'C': 1, 'gamma': 0.001}
✅ Best CV Accuracy: 1.0000


In [21]:
from PIL import Image

def predict_image(image_path, model, encoder):
    img = load_img(image_path, target_size=(224, 224))
    arr = img_to_array(img)
    arr = np.expand_dims(arr, axis=0)

    f1 = mobilenet.predict(mobilenet_preprocess(arr.copy()), verbose=0)
    f2 = resnet.predict(resnet_preprocess(arr.copy()), verbose=0)
    fused = np.concatenate([f1.flatten(), f2.flatten()]).reshape(1, -1)

    fused_scaled = scaler.transform(fused)
    pred = model.predict(fused_scaled)[0]
    label = encoder.inverse_transform([pred])[0]

    print(f"🖼️ Image: {os.path.basename(image_path)}")
    print(f"🔍 Prediction: {label.upper()}")

    img = Image.open(image_path)
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"Prediction: {label}", color="green" if label == 'healthy' else "red")
    plt.show()

In [ ]:
# 📂 Replace with the full path to your test image
image_path = input("Enter the full path to your test image: ")

# 👇 Predict using your SVM model
predict_image(image_path, grid.best_estimator_, encoder)
